In [7]:
import pandas as pd
import numpy as np
import pickle
import cornac
from cornac.data import ImageModality
from cornac.eval_methods import RatioSplit
import torch

In [2]:
items = pd.read_parquet('../data/processed/items.parquet.gzip')
users = pd.read_parquet('../data/processed/users.parquet.gzip')
interactions = pd.read_parquet('../data/processed/interactions.parquet.gzip')

In [6]:
items_embeddings = torch.load('items_embeddings_resnet.pt')

In [20]:
items_embeddings = pd.DataFrame.from_dict(items_embeddings).T
items_embeddings.head()

,0,1,2,3,4,5,6,7,8,9,...,502,503,504,505,506,507,508,509,510,511
10-196395451,0.237612,2.012158,0.592478,0.503925,0.324462,0.996493,0.391664,0.828490,1.718607,0.702388,...,0.600459,0.346801,2.479350,2.099048,0.693619,0.740939,1.357664,0.746173,0.861247,0.304335
10002-196395451,0.926780,1.591932,0.694909,0.388574,0.258138,1.304069,1.277198,0.961014,1.131047,0.768487,...,0.510640,0.915689,1.472894,1.082744,1.349759,1.389173,1.249130,0.643406,0.052001,1.024601
100030-194512826,1.251097,0.187556,1.175913,0.799209,0.219866,0.825076,1.094778,0.176478,1.346639,0.877019,...,0.237150,0.164896,1.697968,0.398451,0.380255,0.432491,0.543562,1.672053,0.354373,0.459834
100045-194512826,1.135396,0.738790,1.181271,0.796464,0.757558,1.229621,1.449003,0.969549,1.709757,0.849875,...,1.376943,0.916532,1.198239,1.930800,1.031750,1.117692,0.822726,1.160130,0.685899,0.745533
100056-194512826,0.304884,1.386500,0.827458,0.753109,0.273594,1.625512,0.356396,0.425700,1.966992,0.680229,...,0.597238,0.454717,1.481134,1.010838,2.186029,0.417488,1.215904,1.908633,0.739498,0.513064


In [23]:
embeddings = []
for i in range(items_embeddings.shape[0]):
    embeddings.append(items_embeddings.iloc[i].values)

In [3]:
feedback = list(interactions.itertuples(index=False, name=None))

In [30]:
item_image_modality = ImageModality(features=np.array(embeddings), 
                                    ids=list(items_embeddings.index),
                                    normalized=True)

In [31]:
ratio_split = RatioSplit(
    data=feedback,
    test_size=0.1,
    val_size=0.15,
    exclude_unknowns=True,
    verbose=True,
    seed=123,
    item_image=item_image_modality
)

rating_threshold = 1.0
exclude_unknowns = True


c:\Users\local\Documents\Roma_thesis\StyleSocial\venv\lib\site-packages\cornac\data\dataset.py:361: UserWarning: 5029 duplicated observations are removed!
  warnings.warn("%d duplicated observations are removed!" % dup_count)


---
Training data:
Number of users = 67454
Number of items = 20191
Number of ratings = 3363663
Max rating = 1.0
Min rating = 1.0
Global mean = 1.0


c:\Users\local\Documents\Roma_thesis\StyleSocial\venv\lib\site-packages\cornac\data\dataset.py:361: UserWarning: 95 duplicated observations are removed!
  warnings.warn("%d duplicated observations are removed!" % dup_count)


---
Test data:
Number of users = 53543
Number of items = 20157
Number of ratings = 449010
Number of unknown users = 0
Number of unknown items = 0


c:\Users\local\Documents\Roma_thesis\StyleSocial\venv\lib\site-packages\cornac\data\dataset.py:361: UserWarning: 205 duplicated observations are removed!
  warnings.warn("%d duplicated observations are removed!" % dup_count)


---
Validation data:
Number of users = 59044
Number of items = 20187
Number of ratings = 673468
---
Total users = 67454
Total items = 20191


In [35]:
# Instantiate evaluation metrics
ndcg_50 = cornac.metrics.NDCG(k=50)
rec_50 = cornac.metrics.Recall(k=50)
prec_50 = cornac.metrics.Precision(k=50)
map_metr = cornac.metrics.MAP()
metrics = [ndcg_50, rec_50, prec_50, map_metr]

vbpr = cornac.models.VBPR(
        k=35,
        k2=35,
        n_epochs=50,
        batch_size=128,
        learning_rate=0.0008,
        lambda_w=0.01,
        lambda_b=0.01,
        lambda_e=0.0001,
        use_gpu=True,
        trainable=True,
        verbose=True,
        seed=42
        )

causalrec = cornac.models.CausalRec(
    k=35,
    k2=35,
    n_epochs=50,
    batch_size=128,
    learning_rate=0.0008,
    lambda_w=1,
    lambda_b=0.01,
    lambda_e=0.0001,
    mean_feat=np.array(embeddings).mean(),
    tanh=0,
    lambda_2=0.5,
    use_gpu=True,
    seed=42
)


models = [vbpr, causalrec]

In [36]:
cornac.Experiment(
    eval_method=ratio_split, 
    models=models, 
    metrics=metrics
).run()


[VBPR] Training started!


Epoch 1/50:   0%|          | 0/26279 [00:00<?, ?it/s]

Epoch 2/50:   0%|          | 0/26279 [00:00<?, ?it/s]

Epoch 3/50:   0%|          | 0/26279 [00:00<?, ?it/s]

Epoch 4/50:   0%|          | 0/26279 [00:00<?, ?it/s]

...

Epoch 48/50:   0%|          | 0/26279 [00:00<?, ?it/s]

Epoch 49/50:   0%|          | 0/26279 [00:00<?, ?it/s]

Epoch 50/50:   0%|          | 0/26279 [00:00<?, ?it/s]

Optimization finished!


RuntimeError: self must be a matrix

In [58]:
causalrec = cornac.models.CausalRec(
    k=45,
    k2=45,
    n_epochs=30,
    batch_size=128,
    learning_rate=0.005,
    lambda_w=1,
    lambda_b=0.01,
    lambda_e=0.0,
    mean_feat=np.array(embeddings).mean(axis=0),
    tanh=0,
    lambda_2=0.5,
    use_gpu=True,
    seed=42
)

cornac.Experiment(
    eval_method=ratio_split, 
    models=[causalrec], 
    metrics=metrics
).run()


[CausalRec] Training started!


Epoch 1/30:   0%|          | 0/26279 [00:00<?, ?it/s]

Epoch 2/30:   0%|          | 0/26279 [00:00<?, ?it/s]

Epoch 3/30:   0%|          | 0/26279 [00:00<?, ?it/s]

Epoch 4/30:   0%|          | 0/26279 [00:00<?, ?it/s]

...

Epoch 30/30:   0%|          | 0/26279 [00:00<?, ?it/s]

Optimization finished!

[CausalRec] Evaluation started!


Ranking:   0%|          | 0/53543 [00:00<?, ?it/s]

Ranking:   0%|          | 0/59044 [00:00<?, ?it/s]


VALIDATION:
...
          |    MAP | NDCG@50 | Precision@50 | Recall@50 |  Time (s)
--------- + ------ + ------- + ------------ + --------- + ---------
CausalRec | 0.0007 |  0.0014 |       0.0006 |    0.0030 | 1142.2080

TEST:
...
          |    MAP | NDCG@50 | Precision@50 | Recall@50 |  Train (s) |  Test (s)
--------- + ------ + ------- + ------------ + --------- + ---------- + ---------
CausalRec | 0.0006 |  0.0013 |       0.0005 |    0.0029 | 16667.2453 | 1016.4619



In [53]:
vbpr2 = cornac.models.VBPR(n_epochs=1).load('../models/VBPR/')

In [54]:
cornac.Experiment(
    eval_method=ratio_split, 
    models=[vbpr2], 
    metrics=metrics
).run()


[VBPR] Training started!

[VBPR] Evaluation started!


Ranking:   0%|          | 0/53543 [00:00<?, ?it/s]

Ranking:   0%|          | 0/59044 [00:00<?, ?it/s]


VALIDATION:
...
     |    MAP | NDCG@50 | Precision@50 | Recall@50 | Time (s)
---- + ------ + ------- + ------------ + --------- + --------
VBPR | 0.0191 |  0.0418 |       0.0132 |    0.0924 | 408.0950

TEST:
...
     |    MAP | NDCG@50 | Precision@50 | Recall@50 | Train (s) | Test (s)
---- + ------ + ------- + ------------ + --------- + --------- + --------
VBPR | 0.0170 |  0.0376 |       0.0098 |    0.0914 |    0.0520 | 375.8450

